In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import OrdinalEncoder
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
import optuna
from sklearn.metrics import accuracy_score
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

c:\Users\Andrey\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [22]:
df = pd.read_csv("C:/Users/Andrey/Documents/Ml/train.csv")

In [3]:
'''
Наименование поля (признака)	Описание
slctn_nmbr	Номер выборки
client_id	ID клиента
npo_account_id	ID счета клиента
npo_accnts_nmbr	Число счетов клиента
pmnts_type	Тип взносов
0: не определен,
1: смешанные,
2: через банк,
3: от работодателя
year	Год агрегации признаков
quarter	Квартал агрегации признаков
gender	Пол (-1: ж, 1: м)
age	Возраст клиента в каждом квартале/годе
clnt_cprtn_time_d	Время «жизни» клиента в днях:
- для действующих клиентов: разница между концом квартала и датой регистрации счета;
- для ушедших клиентов: разница между датой ухода и датой регистрации счета
actv_prd_d	Активный период в днях: разница между датой последнего взноса и датой первого взноса
lst_pmnt_rcnc_d	Давность предыдущего взноса (в днях):
- если взнос есть: разница между датой первого взноса в квартале и датой последнего взноса в предыдущем квартале
- если взноса нет: разница между датой конца квартала и датой последнего взноса в предыдущем квартале
balance	Баланс счета на конец квартала (накопительным итогом) с учетом суммы на счете до перехода клиента в текущий НПФ
oprtn_sum_per_qrtr	Сумма операций по счету в квартале без учета суммы на счете до перехода клиента в текущий НПФ
oprtn_sum_per_year	Сумма операций по счету в без учета суммы на счете до перехода клиента в текущий НПФ
frst_pmnt_date	Дата первого взноса
lst_pmnt_date_per_qrtr	Дата последнего взноса в квартале
frst_pmnt	Размер первого взноса
lst_pmnt	Размер последнего взноса
pmnts_sum	Сумма взносов на конец квартала (накопительным итогом) без учета суммы на счете до перехода клиента в текущий НПФ
pmnts_nmbr	Число взносов на конец квартала (накопительным итогом)
pmnts_sum_per_qrtr	Сумма НПО взносов в квартале
pmnts_sum_per_year	Сумма НПО взносов в году
pmnts_nmbr_per_qrtr	Число НПО взносов за квартал
pmnts_nmbr_per_year	Число НПО взносов за год
incm_sum	Сумма инвест-дохода (ИД) на конец квартала (накопительным итогом) с учетом суммы ИД на счете до перехода клиента в текущий НПФ
incm_per_qrtr	Сумма ИД за квартал с учетом суммы ИД на счете до перехода клиента в текущий НПФ
incm_per_year	Сумма ИД за год с учетом суммы ИД на счете до перехода клиента в текущий НПФ
mgd_accum_period	Минимальный гарантированный доход (МГД) на этапе накопления
mgd_payment_period	МГД на этапе выплат
churn	Маркер действующего (0) / ушедшего по собственному желанию (1) клиента
phone_number	Маркер наличия номера телефона (1: да, -1: нет, 0: нет данных)
email	Маркер наличия email (1: да, -1: нет, 0: нет данных)
lk	Маркер наличия ЛК (1: да, -1: нет, 0: нет данных)
assignee_npo	Маркер является ли клиент приемником НПО счета (1: да, -1: нет, 0: нет данных)
assignee_ops	Маркер является ли клиент приемником ОПС счета (1: да, -1: нет, 0: нет данных)
postal_code	Почтовый индекс, указанный при заключении договора
region	Регион, указанный при заключении договора
citizen	Маркер является ли клиент городским жителем (1: да, -1: нет, 0: нет данных)
fact_addrss	Маркер является ли указанный адрес адресом фактического прожив. клиента (1: да, -1: нет, 0: нет данных)
appl_mrkr	Маркер были ли жалобы у клиента (0: нет, 1: да)
evry_qrtr_pmnt	Маркер платит ли клиент каждый квартал или нет (0: нет, 1: да)
'''
df.head()

,slctn_nmbr,client_id,npo_account_id,npo_accnts_nmbr,pmnts_type,year,quarter,gender,age,clnt_cprtn_time_d,...,lk,assignee_npo,assignee_ops,postal_code,region,citizen,fact_addrss,appl_mrkr,evry_qrtr_pmnt,churn
0,1,0xA095932790098744A2325A8D152C05C7,0xD4DBBAC6561929409BA45725A220613E,1,2,2017,2017Q4,1,48,8091,...,-1,-1,-1,446254.0,САМАРСКАЯ ОБЛ,-1,-1,0,0,0
1,0,0xF75BAA20A603A2479005DEB80464336E,0x3DCFF11974B3D8458CE3569DE9E855D5,2,2,2009,2009Q1,1,35,3303,...,-1,-1,-1,399841.0,ЛИПЕЦКАЯ ОБЛ,-1,1,0,1,0
2,2,0x0D18081E0556AF4F95A931FBA4159D52,0x9DADF88CB3407C4E89403315F640393E,1,1,2010,2010Q1,1,47,1163,...,-1,-1,-1,302038.0,ОРЛОВСКАЯ ОБЛ,1,1,0,1,0
3,0,0x0AD72BC5475D9A4D83E12350A84F839D,0xF59C4E62F115684899EAE9B5D5ECF70F,1,1,2005,2005Q4,-1,42,47,...,-1,-1,-1,620034.0,СВЕРДЛОВСКАЯ ОБЛ,1,1,0,1,0
4,0,0x73DF06339CC25840838408BAD52BE223,0x35BFFA05321C014B9FC29A189E89EDF0,2,2,2007,2007Q4,1,42,876,...,-1,-1,-1,398035.0,ЛИПЕЦКАЯ ОБЛ,1,1,0,1,0


In [4]:
''' Проверка на наличие отсутствующих данных в других записях этого же клиента
client_first_payment_dates = df.groupby('client_id')['frst_pmnt_date'].agg(list)

# Проверка наличия пустых значений
empty_values = client_first_payment_dates.isnull().any()

print(client_first_payment_dates)
print("Есть ли пустые значения:", empty_values)
'''

' Проверка на наличие отсутствующих данных в других записях этого же клиента\nclient_first_payment_dates = df.groupby(\'client_id\')[\'frst_pmnt_date\'].agg(list)\n\n# Проверка наличия пустых значений\nempty_values = client_first_payment_dates.isnull().any()\n\nprint(client_first_payment_dates)\nprint("Есть ли пустые значения:", empty_values)\n'

In [23]:
df = df.drop(['slctn_nmbr','postal_code', 'region', 'phone_number', 'email', 'lst_pmnt_date_per_qrtr'], axis=1)

df['frst_pmnt_date'] = df.groupby('client_id')['frst_pmnt_date'].transform(lambda x: x.fillna(method='ffill').fillna(method='bfill'))
df = df.dropna(subset = 'frst_pmnt_date')

C:\Users\Andrey\AppData\Local\Temp\ipykernel_13888\953680723.py:3: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['frst_pmnt_date'] = df.groupby('client_id')['frst_pmnt_date'].transform(lambda x: x.fillna(method='ffill').fillna(method='bfill'))
C:\Users\Andrey\AppData\Local\Temp\ipykernel_13888\953680723.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['frst_pmnt_date'] = df.groupby('client_id')['frst_pmnt_date'].transform(lambda x: x.fillna(method='ffill').fillna(method='bfill'))


In [6]:
empty_columns = df.isnull().sum()
print(empty_columns[empty_columns > 0])

Series([], dtype: int64)


In [24]:
df[['first_pmnt_year', 'first_pmnt_month', 'first_pmnt_day']] = df['frst_pmnt_date'].str.split('-', expand=True)
df['first_pmnt_year'] = df['first_pmnt_year'].astype(int)
df['first_pmnt_month'] = df['first_pmnt_month'].str.lstrip('0').astype(int)
df['first_pmnt_day'] = df['first_pmnt_day'].str.lstrip('0').astype(int)

In [8]:
df.dtypes

client_id               object
npo_account_id          object
npo_accnts_nmbr          int64
pmnts_type               int64
year                     int64
quarter                 object
gender                   int64
age                      int64
clnt_cprtn_time_d        int64
actv_prd_d               int64
lst_pmnt_rcnc_d          int64
balance                float64
oprtn_sum_per_qrtr     float64
oprtn_sum_per_year     float64
frst_pmnt_date          object
frst_pmnt              float64
lst_pmnt               float64
pmnts_sum              float64
pmnts_nmbr               int64
pmnts_sum_per_qrtr     float64
pmnts_sum_per_year     float64
pmnts_nmbr_per_qrtr      int64
pmnts_nmbr_per_year      int64
incm_sum               float64
incm_per_qrtr          float64
incm_per_year          float64
mgd_accum_period       float64
mgd_payment_period     float64
lk                       int64
assignee_npo             int64
assignee_ops             int64
citizen                  int64
fact_add

In [25]:
df['quarter'] = df['quarter'].str[-1].astype(int)
heat_x = df.drop(['client_id', 'npo_account_id', 'frst_pmnt_date'], axis=1)
''' график корреляции
correlations = heat_x.corrwith(heat_x['churn'])

# Создаем DataFrame из корреляций
correlation_df = correlations.to_frame(name='correlation_with_churn')

# Строим тепловую карту для визуализации корреляций с 'churn'
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_df, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation with Churn')
plt.show()
'''

' график корреляции\ncorrelations = heat_x.corrwith(heat_x[\'churn\'])\n\n# Создаем DataFrame из корреляций\ncorrelation_df = correlations.to_frame(name=\'correlation_with_churn\')\n\n# Строим тепловую карту для визуализации корреляций с \'churn\'\nplt.figure(figsize=(10, 8))\nsns.heatmap(correlation_df, annot=True, cmap=\'coolwarm\', fmt=".2f")\nplt.title(\'Correlation with Churn\')\nplt.show()\n'

In [26]:
y = heat_x.pop('churn')
clf = RandomForestClassifier(n_estimators=3, random_state=42)
clf.fit(heat_x, y)
feature_importance_df = pd.DataFrame({'Feature': heat_x.columns, 'Importance': clf.feature_importances_})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)
feature_importance_df.head(50)

,Feature,Importance
11,oprtn_sum_per_year,0.162162
22,incm_per_year,0.074522
19,pmnts_nmbr_per_year,0.070378
3,quarter,0.064200
10,oprtn_sum_per_qrtr,0.059650
9,balance,0.044037
15,pmnts_nmbr,0.038389
17,pmnts_sum_per_year,0.037983
8,lst_pmnt_rcnc_d,0.033520
13,lst_pmnt,0.032951


In [30]:
X_train, X_test, y_train, y_test = train_test_split(heat_x, y, test_size=0.3, random_state=41)
def objective(trial):
    params = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'max_depth': trial.suggest_int('max_depth', 2, 30),
        'min_child_samples': trial.suggest_int('min_child_samples', 1, 20),
        # Другие параметры, которые вы хотите оптимизировать
    }
    model = LGBMClassifier(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    score = accuracy_score(y_test, y_pred)
    return score

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)
print(study.best_params)
print(study.best_value)


[I 2024-04-13 10:35:24,971] A new study created in memory with name: no-name-97354efd-dadd-417b-af52-e9efab90d594
[I 2024-04-13 10:36:06,730] Trial 0 finished with value: 0.9927051017639001 and parameters: {'max_depth': 18, 'min_child_samples': 16}. Best is trial 0 with value: 0.9927051017639001.
[W 2024-04-13 10:36:11,355] Trial 1 failed with parameters: {'max_depth': 25, 'min_child_samples': 11} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\Andrey\AppData\Local\Programs\Python\Python312\Lib\site-packages\optuna\study\_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\Andrey\AppData\Local\Temp\ipykernel_13888\677540353.py", line 16, in objective
    scores = cross_val_score(model, X_test, y_test, cv=5)  # 5 фолдов для кросс-валидации
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Andrey\AppData\Local\Programs\Python\Python312\L

KeyboardInterrupt: 